# COLLECTING IMDBIDS AND TITLES
---

Nhiệm vụ phần này: từ danh sách các đường dẫn phim của trang web `Just Watch`, thực hiện cào thông tin **imdbId** và **title** trong đường dẫn nếu không có id và lưu lại vào file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [43]:
import time
import os
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

## 2. CÀO THÔNG TIN IMDBID TRONG TRANG `JUST WATCH`

In [28]:
links = []

with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

Dùng selenium thoi vì request k được

In [29]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 629,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 1259,
        "end": 1887
    } 
}

In [30]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

In [56]:
def collect_imdbID_title(links, start, end):
    
    option = webdriver.ChromeOptions()

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.plugins": 2,
        "profile.managed_default_content_settings.media": 2
    }
    option.add_experimental_option("prefs", prefs)
    # option.page_load_strategy = 'eager'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    imdbIds_titles = []

    for url in links[start:end + 1]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        time.sleep(3)
        page_source = driver.page_source

        imdb_id_matches = re.findall(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_matches:
            imdb_id = imdb_id_matches[-1]
            imdbIds_titles.append(imdb_id)
            print("IMDB ID:", imdb_id)
        else:
            # print("Không tìm thấy IMDb ID.")
            match = re.search(r'class="release-year".*?\((\d{4})\)', page_source)
            release_year = match.group(1)
            movie_title_meta = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            if movie_title_meta:
                movie_title = movie_title_meta.get_attribute("content").split(" streaming")[0]
                movie_title = f"{movie_title} __ {release_year}"
            else:
                movie_title = "Null"
            imdbIds_titles.append(movie_title)
            print(movie_title)

    driver.quit()
    return imdbIds_titles

In [57]:
imdbIds_titles = collect_imdbID_title(links, start=start, end=end)
imdbIds_titles

[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-mission-2023
IMDB ID: tt28097834
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-peasants
IMDB ID: tt10651230
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-burial-0
The Burial __ 2023
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/saw-x
Saw X __ 2023
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/last-call-for-istanbul
IMDB ID: tt23334616
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/coup-de-chance
IMDB ID: tt15140278
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/shooting-stars-2023
IMDB ID: tt3715152
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/wingwomen
IMDB ID: tt21638298
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/love-in-the-maldives
IMDB ID: tt26938504
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/one-day-as-a-lion
IMDB ID: tt15783426
[LOG]: parse imdbid of url https

KeyboardInterrupt: 

In [49]:
# chiến lược tải trang nhanh hơn

options = webdriver.ChromeOptions()

prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.media": 2
}
options.add_experimental_option("prefs", prefs)
options.page_load_strategy = 'eager' # có vẻ như lệnh này nhanh hơn trong máy mình, các bạn thử uncoment thử coi sao chứ 

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
url = 'https://www.justwatch.com/us/movie/the-mission-2023'
driver.get(url)
time.sleep(5)
page_source = driver.page_source

imdb_id_match = re.findall(r'"imdbId":"(tt\d+)"', page_source)
if imdb_id_match:
    imdb_id = imdb_id_match[-1]
    #imdbIds_titles.append(imdb_id)
    print("IMDB ID:", imdb_id)
else:
    print("Không tìm thấy IMDb ID.")

driver.quit()

IMDB ID: tt28097834


In [39]:
import re

# Ví dụ chuỗi có chứa nhiều giá trị trong ngoặc đơn
page_source = '"imdbId":"(tt1234567)", "imdbId":"(tt7654321)", "imdbId":"(tt9876543)"'

# Sử dụng re.findall để tìm tất cả các giá trị trong ngoặc đơn
matches = re.findall(r'"imdbId":"\((tt\d+)\)"', page_source)

# Kiểm tra và lấy giá trị trong ngoặc đơn của chuỗi cuối cùng
if matches:
    last_imdb_id = matches[-1]  # Lấy phần tử cuối cùng trong danh sách
    print(f"Last IMDb ID: {last_imdb_id}")
else:
    print("No IMDb ID found.")

Last IMDb ID: tt9876543


## 3. LƯU THÔNG TIN VỀ ID VÀ TITLE VÀO FILE TXT

In [ ]:
file_name = os.path.join("./Data/", f"ids_title.csv")

with open(file_name, 'a') as f:
    for item in imdbIds_titles:
        f.write(item + "\n")